In [1]:
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv
load_dotenv()
pd.options.display.max_columns = None

In [2]:
#pip install cryptography
#pip show pymysql

In [3]:
df = pd.read_csv("productos.csv")

In [4]:
#defino los parámetros para la conexión desde el archivo .env
username = os.getenv("username")
password = os.getenv("password")
host = os.getenv("host")
database = os.getenv("database")

# Crear la cadena de conexión
connection_string = f'mysql+pymysql://{username}:{password}@{host}/{database}'
engine = create_engine(connection_string)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3871 entries, 0 to 3870
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   categoria     3871 non-null   object 
 1   subcategoria  3871 non-null   object 
 2   nombre        3871 non-null   object 
 3   precio        3856 non-null   float64
 4   url           3871 non-null   object 
 5   tienda        3871 non-null   object 
 6   peso          1463 non-null   object 
 7   chamota       49 non-null     object 
 8   tipo          181 non-null    object 
 9   capacidad     372 non-null    object 
 10  tamaño        532 non-null    object 
dtypes: float64(1), object(10)
memory usage: 332.8+ KB


In [6]:
#añado las columnas que serán las claves primarias de mis tablas
df['id_tienda'] = df.groupby('tienda').ngroup() + 1
df['id_categoria'] = df.groupby('categoria').ngroup() + 1
df['id_subcategoria'] = df.groupby('subcategoria').ngroup() + 1
df['id_producto'] = df.groupby('nombre').ngroup() + 1

In [7]:
# genero un dataframe por cada tabla que va a tener mi bbdd con las colunas que tendrá cada una
df_tiendas = df[["id_tienda", "tienda"]].drop_duplicates().reset_index(drop=True)
df_categorias = df[["id_categoria", "categoria"]].drop_duplicates().reset_index(drop=True)
df_subcategorias = df[["id_subcategoria", "subcategoria", "id_categoria"]].drop_duplicates().reset_index(drop=True)
df_productos = df[["id_producto", "nombre", "precio", "url", "peso", "chamota", "tipo", "capacidad", "tamaño",  "id_categoria", "id_subcategoria", "id_tienda"]].drop_duplicates().reset_index(drop=True)

In [ ]:
# creo un diccionario para modificar los nombres de las columnas y hacer que coincidan con los que hay en la bbdd
column_mapping = {
    'id_categoria': 'id_categoria',
    'categoria': 'nombre_categoria',
}

df_categorias = df_categorias.rename(columns=column_mapping)

# Insertar los datos en la tabla 'categorias'
df_categorias.to_sql(name='categorias', con=engine, if_exists='append', index=False)

In [ ]:
# creo un diccionario para modificar los nombres de las columnas y hacer que coincidan con los que hay en la bbdd
df_subcategorias.drop_duplicates(subset=['id_subcategoria'], keep='first', inplace=True)
column_mapping = {
    'id_subcategoria': 'id_subcategoria',
    'subcategoria': 'nombre_subcategoria',
    'id_categoria': 'id_categoria'
}

df_subcategorias = df_subcategorias.rename(columns=column_mapping)

# Insertar los datos en la tabla 'subcategorias'
df_subcategorias.to_sql(name='subcategorias', con=engine, if_exists='append', index=False)

133

In [ ]:
# creo un diccionario para modificar los nombres de las columnas y hacer que coincidan con los que hay en la bbdd
df_tiendas.drop_duplicates(subset=['id_tienda'], keep='first', inplace=True)
column_mapping = {
    'id_tienda': 'id_tienda',
    'tienda': 'nombre_tienda'
}

df_tiendas = df_tiendas.rename(columns=column_mapping)

# Insertar los datos en la tabla 'tiendas'
df_tiendas.to_sql(name='tiendas', con=engine, if_exists='append', index=False)

3

In [46]:
# creo un diccionario para modificar los nombres de las columnas y hacer que coincidan con los que hay en la bbdd
df_productos.drop_duplicates(subset=['id_producto'], keep='first', inplace=True)

column_mapping = {
    'id_producto': 'id_producto',
    'nombre': 'nombre_producto',
    'precio': 'precio',
    'url': 'url',
    'peso': 'peso',
    'chamota': 'chamota',
    'tipo': 'tipo',
    'capacidad': 'capacidad',
    'tamaño': 'dimensiones',
    'id_subcategoria': 'id_subcategoria',
    'subcategoria': 'nombre_subcategoria',
    'id_categoria': 'id_categoria'
}

df_productos = df_productos.rename(columns=column_mapping)

# Insertar los datos en la tabla 'productos'
df_productos.to_sql(name='productos', con=engine, if_exists='append', index=False)

3199